# Analyse time-dependent proteome

## 1. Load data into workbook

In [3]:
import cPickle as pkl
import re
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.nonparametric.smoothers_lowess import lowess

Write part of the directory listing to a variable:

In [16]:
files = !ls ..\results\po*

In [17]:
ls ..\results\po*

 Datentr„ger in Laufwerk C: ist Windows8_OS
 Volumeseriennummer: 54DB-BDA9

 Verzeichnis von C:\Users\MJS\git\TRSL1\results

26.01.2016  15:09        19.291.652 polyphasic cell cycle from 0 to 300, updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20160121_1034_0600s.p
26.01.2016  15:08        34.950.891 polyphasic cell cycle from 1200 to 1500, updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20160121_1103_0600s.p
26.01.2016  15:08        42.684.310 polyphasic cell cycle from 1500 to 1800, updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20160121_1112_0600s.p
26.01.2016  15:08        43.574.465 polyphasic cell cycle from 1800 to 2100, updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20160121_1121_0600s.p
26.01.2016  15:09        43.948.524 polyphasic cell cycle from 2100 to 2400, updated Shah transcriptome, full e

Set up a dictionary that contains the simulation results.

In [19]:
filepaths = {}
for file in files:
    match = re.search(r'from (\w+)', file)
    print match
    filepaths[int(match.group(1).zfill(4))] = file

None


AttributeError: 'NoneType' object has no attribute 'group'

Function to replace _numpypy.multiarray by numpy.core.multiarray. Source: http://stackoverflow.com/questions/39086/search-and-replace-a-line-in-a-file-in-python

In [30]:
from tempfile import mkstemp
from shutil import move
from os import remove, close

def replace(file_path, pattern, subst):
    #Create temp file
    fh, abs_path = mkstemp()
    with open(abs_path,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(pattern, subst))
    close(fh)
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)

Create proteome dictionary:

In [35]:
results = {}
proteome = {}
for time in filepaths:
    print time,
    replace(filepaths[time], "_numpypy.multiarray", "numpy.core.multiarray")
    results[time] = pkl.load(open(filepaths[time]))
    proteome[time] = {key: results[time]['proteome'][key] for key in results[time]['proteome'] 
                     if results[time]['proteome'][key] > 1e-1}

0 2400 3300 1800 300 2700 1200 600 2100 3000 900 1500


## 2. Test whether burn-in period has been long enough

In [36]:
def scatter_dicts(dict1, dict2, xaxislabel=None, yaxislabel=None, fontsize=None):
    plotkeys = dict1.viewkeys() & dict2.viewkeys() # set intersection
    print len(plotkeys)
    xs = [dict1[key] for key in plotkeys]
    ys = [dict2[key] for key in plotkeys] 

    fig =  plt.figure()                

    ax = fig.add_subplot(111)
    ax.grid(True, linestyle = '-', color = '0.75')
    ax.set_xscale('log')
    ax.set_yscale('log')
    if xaxislabel:
        ax.set_xlabel(xaxislabel)
        if fontsize:
            ax.set_xlabel(xaxislabel, fontsize=fontsize)
            plt.setp(ax.get_xticklabels(), fontsize=fontsize)
    if yaxislabel:
        ax.set_ylabel(yaxislabel)
        if fontsize:
            ax.set_ylabel(yaxislabel, fontsize=fontsize)
            plt.setp(ax.get_yticklabels(), fontsize=fontsize)

    scat = plt.scatter(xs, ys)
    scat.set_alpha(0.2)

    points = zip(xs, ys)
    sorted_points = sorted(points)
    new_xs = [point[0] for point in sorted_points]
    new_ys = [point[1] for point in sorted_points]
    ylowess = lowess(new_ys, new_xs)[:,1]
    plt.plot(new_xs, ylowess, 'orange', linewidth=4)

    plt.show()

In [37]:
scatter_dicts(proteome[0], proteome[3300], xaxislabel="proteome (300 s)", yaxislabel="proteome (3600 s)", fontsize=None)

3123
